<a href="https://colab.research.google.com/github/eduardax/imbalanced-data-analysis/blob/main/vetorization_oversampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalando e Importando
!pip install pandas-profiling
!pip install imbalanced-learn
import imblearn
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.datasets import make_classification
import os
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, RandomOverSampler, ADASYN
import warnings
import traceback
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from imblearn.pipeline import make_pipeline
import matplotlib
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from imblearn import FunctionSampler  # to use a idendity sampler

%matplotlib inline

print(__doc__)

sns.set_context("poster")

In [ ]:
# Montando a partição do Google Drive para acessar os arquivos
from google.colab import drive
drive.mount('/content/drive', True)

In [ ]:
# Função para carregar dados
def load_data(file_path):
    df = pd.read_csv(file_path, sep=';', encoding='utf-8', usecols=['label', 'processed_abstract_text'])
    df['label'] = df['label'] - 1  # Ajusta as labels para começar de 0
    return df

arquivo = '/content/drive/MyDrive/Colab Notebooks/tcc_pro/tcc/original_and_preprocessed.csv'

dataset = load_data(arquivo)
dataset = pd.DataFrame(dataset)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Extrair as informações do CSV
labels = dataset['label'].values + 1

fig, ax = plt.subplots(figsize=(16, 7))
idx, c = np.unique(labels, return_counts=True)

# Definir as cores para cada ODS
ods_colors = {
    1: "#E5243B",
    2: "#F4F5F6",
    3: "#4C9F38",
    4: "#C5192D",
    5: "#FF3A21",
    6: "#26BDE2",
    7: "#FCC30B",
    8: "#A21942",
    9: "#FD6925",
    10: "#DD1367",
    11: "#FD9D24",
    12: "#BF8B2E",
    13: "#3F7E44",
    14: "#0A97D9",
    15: "#56C02B",
    16: "#00689D",
    17: "#19486A"
}

# Criar a lista de cores baseado nos labels
colors = [ods_colors.get(label, "#fffff") for label in idx]  # Azul padrão para valores não especificados

# Plotar o gráfico de barras
sns.barplot(x=idx, y=c, ax=ax, palette=colors)

# Adicionar valores no topo de cada barra com tamanho de fonte ajustado
for i in range(len(idx)):
    ax.text(i, c[i], str(c[i]), ha='center', va='bottom', fontsize=14)

plt.tight_layout()

# Salvar a imagem plotada em um diretório específico
output_directory = '/content/drive/MyDrive/Colab Notebooks/tcc_pro/tcc/'
output_filename = 'class_distribution_BEFORE_oversampling.png'
output_path = f"{output_directory}/{output_filename}"
plt.savefig(output_path)

plt.show()


In [ ]:
X = dataset['processed_abstract_text']
y = dataset['label']
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=42)

num_train_samples = len(X_train)
num_train_labels = len(y_train)
num_test_samples = len(X_test)
num_test_labels = len(y_test)

print(f"Quantidade de exemplos em X_train: {num_train_samples}")
print(f"Quantidade de rótulos em y_train: {num_train_labels}")
print(f"Quantidade de exemplos em X_test: {num_test_samples}")
print(f"Quantidade de rótulos em y_test: {num_test_labels}")

In [ ]:
# Criar DataFrame com X_sm como uma coluna única
df_test = pd.DataFrame({'processed_abstract_text': X_test, 'label': y_test})
df = pd.DataFrame({'processed_abstract_text': X_train, 'label': y_train})

# Salvar em um arquivo CSV
df_test.to_csv('/content/drive/MyDrive/Colab Notebooks/tcc_pro/tcc/df_test.csv', index=False)

In [ ]:
#Funcao para coletar até 400 samples de cada label
def load_and_select_data(column_a, column_b, num_samples_per_class=300):
    # Criar um DataFrame vazio para armazenar os dados selecionados
    selected_data = pd.DataFrame(columns=[column_a, column_b])

    # Pegar os valores únicos da coluna de classificação
    unique_classes = df_resampled[column_b].unique()

    # Selecionar 400 valores da coluna A para cada classe na coluna B
    for cls in unique_classes:
        class_subset = df_resampled[df_resampled[column_b] == cls]
        sampled_subset = class_subset.sample(n=min(num_samples_per_class, len(class_subset)), random_state=42)
        selected_data = pd.concat([selected_data, sampled_subset])

    return selected_data
# Exibir os dados selecionados
# print(selected_data)

In [ ]:
fv_vectorizer = CountVectorizer(min_df=1)
X_fv = fv_vectorizer.fit_transform(df['processed_abstract_text']).toarray()

tokens = fv_vectorizer.get_feature_names_out() #Aqui pega os tokens
tokens

In [ ]:
# Contar os valores de cada rótulo
value_counts_selected_data = df['label'].value_counts()
print("\nContagem de valores por rótulo:")
print(value_counts_selected_data)

In [ ]:
tokens = fv_vectorizer.get_feature_names_out() #Aqui pega os tokens
tokens

In [ ]:
X_train = X_fv
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(df['label'])

In [ ]:
print('Resampled dataset shape %s' % Counter(y_train))

In [ ]:
def decodeMatrix(matrix, tokens, labels):
    documents = []
    documents_with_tokens_and_labels = []  # Lista para armazenar documentos com tokens e labels correspondentes

    #Colocar retorno para ser dataframe do jeito que tu quiser (Que o BERT no caso)
    for row, label in zip(matrix, labels):
        selected_tokens = []
        for index, value in enumerate(row):
            if value != 0:  # Aqui pode adicionar um for pra add a quantidade de palavras de acordo com o numero do Saco de Palavras
                selected_tokens.append(tokens[index])

        # Concatenar os tokens selecionados para formar o documento
        document = ' '.join(selected_tokens)

        # Armazenar o documento, seus tokens correspondentes e a label
        documents.append(document)
        documents_with_tokens_and_labels.append({
            'new_text': document,
#            'selected_tokens': selected_tokens,
            'label': label
        })

    return documents, documents_with_tokens_and_labels

## RandomOversampler

In [ ]:
ros = RandomOverSampler(random_state=42)
X_res, y_res = ros.fit_resample(X_train.values.reshape(-1, 1), y_train)
print('Resampled dataset shape %s' % Counter(y_res))

# Converter X_sm para uma lista (coluna única)
X_res_list = X_res.tolist()

# Criar DataFrame com X_sm como uma coluna única
df_resampled = pd.DataFrame({'processed_abstract_text': X_res_list, 'label': y_res})

# Salvar em um arquivo CSV
df_resampled.to_csv('/content/drive/MyDrive/Colab Notebooks/tcc_pro/tcc/random/dados_RO.csv', index=False)

# Contar o número de exemplos por classe após o resample
print('Resampled dataset shape %s' % Counter(y_res))

In [ ]:
ostrezentos = load_and_select_data('processed_abstract_text', 'label')

In [ ]:
print('Resampled dataset shape %s' % Counter(ostrezentos['label']))

In [ ]:
# Criar um DataFrame com os documentos, tokens e labels correspondentes
df_with_tokens_and_labels = pd.DataFrame(ostrezentos)

df_with_tokens_and_labels.head()

# Salvar em um arquivo CSV
df_with_tokens_and_labels.to_csv('/content/drive/MyDrive/Colab Notebooks/tcc_pro/tcc/smote/train_SMOTE_after_decode.csv', index=False)

# Contar o número de exemplos por classe após o resample
print('Resampled dataset shape %s' % Counter(df_with_tokens_and_labels['label']))

## SMOTE

In [ ]:
# Aplicação do SMOTE com um número adequado de vizinhos
k_neighbors = 1

smote = SMOTE(random_state=42, k_neighbors=k_neighbors)
X_sm, y_sm = smote.fit_resample(X_train, y_train)

# Converter X_sm para uma lista (coluna única)
X_sm_list = X_sm.tolist()

# Criar DataFrame com X_sm como uma coluna única
df_resampled = pd.DataFrame({'processed_abstract_text': X_sm_list, 'label': y_sm})

# Salvar em um arquivo CSV
df_resampled.to_csv('/content/drive/MyDrive/Colab Notebooks/tcc_pro/tcc/smote/dados_SMOTE.csv', index=False)

# Contar o número de exemplos por classe após o resample
print('Resampled dataset shape %s' % Counter(y_sm))

In [ ]:
df_resampled.head()

In [ ]:
# Decodificar a matriz com tokens e labels
documents, documents_with_tokens_and_labels = decodeMatrix(df_resampled['processed_abstract_text'], tokens, y_sm)

# Criar um DataFrame com os documentos, tokens e labels correspondentes
df_with_tokens_and_labels = pd.DataFrame(documents_with_tokens_and_labels)

df_with_tokens_and_labels.head()

# Salvar em um arquivo CSV
df_with_tokens_and_labels.to_csv('/content/drive/MyDrive/Colab Notebooks/tcc_pro/tcc/smote/train_SMOTE_after_decode.csv', index=False)

# Contar o número de exemplos por classe após o resample
print('Resampled dataset shape %s' % Counter(df_with_tokens_and_labels['label']))

In [ ]:
for i, x in enumerate(df_with_tokens_and_labels['new_text']):
  if not isinstance(x, str): print(f'nan in {i} : {x}')

In [ ]:
#Funcao para coletar até 400 samples de cada label
def load_and_select_data_sm(column_a, column_b, num_samples_per_class=300):
    # Criar um DataFrame vazio para armazenar os dados selecionados
    selected_data = pd.DataFrame(columns=[column_a, column_b])

    # Pegar os valores únicos da coluna de classificação
    unique_classes = df_with_tokens_and_labels[column_b].unique()

    # Selecionar 400 valores da coluna A para cada classe na coluna B
    for cls in unique_classes:
        class_subset = df_with_tokens_and_labels[df_with_tokens_and_labels[column_b] == cls]
        sampled_subset = class_subset.sample(n=min(num_samples_per_class, len(class_subset)), random_state=42)
        selected_data = pd.concat([selected_data, sampled_subset])

    return selected_data

In [ ]:
df_with_tokens_and_labels = load_and_select_data_sm('new_text', 'label')

# Salvar em um arquivo CSV
df_with_tokens_and_labels.to_csv('/content/drive/MyDrive/Colab Notebooks/tcc_pro/tcc/bsmote/train_BSMOTE_after_decode.csv', index=False)

# Contar o número de exemplos por classe após o resample
print('Resampled dataset shape %s' % Counter(df_with_tokens_and_labels['label']))

In [ ]:
df_with_tokens_and_labels.head()

## Borderline-SMOTE

In [ ]:
Bsm = BorderlineSMOTE(random_state=42,  k_neighbors=k_neighbors)
X_res_Bsm, y_res_Bsm = Bsm.fit_resample(X_train, y_train)

# Converter X_sm para uma lista (coluna única)
X_bsm_list = X_res_Bsm.tolist()

# Criar DataFrame com X_sm como uma coluna única
df_resampled_bsm = pd.DataFrame({'processed_abstract_text': X_bsm_list, 'label': y_res_Bsm})

# Salvar em um arquivo CSV
df_resampled_bsm.to_csv('/content/drive/MyDrive/Colab Notebooks/tcc_pro/tcc/bsmote/dados_BSMOTE_after_oversampling.csv', index=False)

# Contar o número de exemplos por classe após o resample
print('Resampled dataset shape %s' % Counter(y_res_Bsm))

In [ ]:
df_resampled_bsm.head()

In [ ]:
# Decodificar a matriz com tokens e labels
documentsBSM, documents_with_tokens_and_labelsBSM = decodeMatrix(df_resampled_bsm['processed_abstract_text'], tokens, y_res_Bsm)

# Criar um DataFrame com os documentos, tokens e labels correspondentes
df_with_tokens_and_labelsBSM = pd.DataFrame(documents_with_tokens_and_labelsBSM)

df_with_tokens_and_labelsBSM.head()

In [ ]:
for i, x in enumerate(df_with_tokens_and_labelsBSM['new_text']):
  if not isinstance(x, str): print(f'nan in {i} : {x}')

In [ ]:
#Funcao para coletar até 400 samples de cada label
def load_and_select_data_ada(column_a, column_b, num_samples_per_class=300):
    # Criar um DataFrame vazio para armazenar os dados selecionados
    selected_data = pd.DataFrame(columns=[column_a, column_b])

    # Pegar os valores únicos da coluna de classificação
    unique_classes = df_with_tokens_and_labelsBSM[column_b].unique()

    # Selecionar 400 valores da coluna A para cada classe na coluna B
    for cls in unique_classes:
        class_subset = df_with_tokens_and_labelsBSM[df_with_tokens_and_labelsBSM[column_b] == cls]
        sampled_subset = class_subset.sample(n=min(num_samples_per_class, len(class_subset)), random_state=42)
        selected_data = pd.concat([selected_data, sampled_subset])

    return selected_data

In [ ]:
df_with_tokens_and_labelsBSM = load_and_select_data_ada('new_text', 'label')

# Salvar em um arquivo CSV
df_with_tokens_and_labelsBSM.to_csv('/content/drive/MyDrive/Colab Notebooks/tcc_pro/tcc/bsmote/train_BSMOTE_after_decode.csv', index=False)

# Contar o número de exemplos por classe após o resample
print('Resampled dataset shape %s' % Counter(df_with_tokens_and_labelsBSM['label']))

In [ ]:
df_with_tokens_and_labelsBSM.head()

## ADASYN

In [ ]:
adasyn = ADASYN(random_state=42, n_neighbors=1)
X_adasyn, y_adasyn = adasyn.fit_resample(X_train, y_train)

# Converter X_sm para uma lista (coluna única)
X_adasyn_list = X_adasyn.tolist()

# Criar DataFrame com X_sm como uma coluna única
df_resampled_adasyn = pd.DataFrame({'processed_abstract_text': X_adasyn_list, 'label': y_adasyn})

# Salvar em um arquivo CSV
df_resampled_adasyn.to_csv('/content/drive/MyDrive/Colab Notebooks/tcc_pro/tcc/adasyn/train_ADASYN_after_oversampling.csv', index=False)

# Contar o número de exemplos por classe após o resample
print('Resampled dataset shape %s' % Counter(y_adasyn))

In [ ]:
df_resampled_adasyn.head()

In [ ]:
# Decodificar a matriz com tokens e labels
documentsADA, documents_with_tokens_and_labelsADA = decodeMatrix(df_resampled_adasyn['processed_abstract_text'], tokens, y_adasyn)

# Criar um DataFrame com os documentos, tokens e labels correspondentes
df_with_tokens_and_labelsADA_pre = pd.DataFrame(documents_with_tokens_and_labelsADA)

df_with_tokens_and_labelsADA = load_and_select_data_ada('new_text', 'label')

df_with_tokens_and_labelsADA.head()

# Salvar em um arquivo CSV
df_with_tokens_and_labelsADA.to_csv('/content/drive/MyDrive/Colab Notebooks/tcc_pro/tcc/adasyn/train_ADASYN_after_decode.csv', index=False)

# Contar o número de exemplos por classe após o resample
print('Resampled dataset shape %s' % Counter(df_with_tokens_and_labelsADA['label']))

In [ ]:
for i, x in enumerate(df_with_tokens_and_labelsADA['new_text']):
  if not isinstance(x, str): print(f'nan in {i} : {x}')

In [ ]:
df_with_tokens_and_labelsADA.head()